In [1]:
import pandas as pd
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
from sklearn import linear_model
import datetime
import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB

train = pd.read_excel('train.xlsx')

train = train.drop(['DOL', 'DOJ', 'Designation', 'TestOrTrain', 'ID','JobCity'], axis =1) 

In [2]:
def yearReturn(x):
    return x.year

def mean1(col):
    sm = 0
    count = 0;

    for x in col:
        if x != -1:
                sm = sm + x;
                count = count + 1;

                mn = sm / count
    return mn;


def transformPresent(col, val):
    new = []

    for x in col:
        if x == 'present':
                new.append(val);
        else:
                new.append(x);

    return new;


def transform(col, val):
    new = []

    for x in col:
        if x == -1:
                new.append(val);
        else:
                new.append(x);

    return new;

In [3]:
train.head()

,Salary,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,...,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience
0,420000,f,1990-02-19,84.3,"board ofsecondary education,ap",2007,95.8,"board of intermediate education,ap",1141,2,...,-1,-1,-1,-1,-1,0.9737,0.8128,0.5269,1.35490,-0.4455
1,500000,m,1989-10-04,85.4,cbse,2007,85.0,cbse,5807,2,...,-1,-1,-1,-1,-1,-0.7335,0.3789,1.2396,-0.10760,0.8637
2,325000,f,1992-08-03,85.0,cbse,2010,68.2,cbse,64,2,...,-1,-1,-1,-1,-1,0.2718,1.7109,0.1637,-0.86820,0.6721
3,1100000,m,1989-12-05,85.6,cbse,2007,83.6,cbse,6920,1,...,-1,-1,-1,-1,-1,0.0464,0.3448,-0.3440,-0.40780,-0.9194
4,200000,m,1991-02-27,78.0,cbse,2008,76.8,cbse,11368,2,...,-1,-1,-1,-1,-1,-0.8810,-0.2793,-1.0697,0.09163,-0.1295


In [4]:
val = mean1(train.ElectronicsAndSemicon)
train.ElectronicsAndSemicon = transform(train.ElectronicsAndSemicon, val);

val = mean1(train.ComputerProgramming)	    
train.ComputerProgramming = transform(train.ComputerProgramming, val);

val = mean1(train.ComputerScience)	    
train.ComputerScience = transform(train.ComputerScience, val);

val = mean1(train.MechanicalEngg)	    
train.MechanicalEngg = transform(train.MechanicalEngg, val);

val = mean1(train.ElectricalEngg)
train.ElectricalEngg = transform(train.ElectricalEngg, val);

val = mean1(train.TelecomEngg)
train.TelecomEngg = transform(train.TelecomEngg, val);

val = mean1(train.CivilEngg)
train.CivilEngg = transform(train.CivilEngg, val);

val = mean1(train.Domain)    
train.Domain = transform(train.Domain, val);

train['DOB'] = train.DOB.apply(yearReturn)

In [5]:
train.head()

,Salary,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,...,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience
0,420000,f,1990,84.3,"board ofsecondary education,ap",2007,95.8,"board of intermediate education,ap",1141,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.9737,0.8128,0.5269,1.35490,-0.4455
1,500000,m,1989,85.4,cbse,2007,85.0,cbse,5807,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,-0.7335,0.3789,1.2396,-0.10760,0.8637
2,325000,f,1992,85.0,cbse,2010,68.2,cbse,64,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.2718,1.7109,0.1637,-0.86820,0.6721
3,1100000,m,1989,85.6,cbse,2007,83.6,cbse,6920,1,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.0464,0.3448,-0.3440,-0.40780,-0.9194
4,200000,m,1991,78.0,cbse,2008,76.8,cbse,11368,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,-0.8810,-0.2793,-1.0697,0.09163,-0.1295


In [6]:
text_columns = []
for f in train.columns:
    if train[f].dtype=='object':
        text_columns.append(f)            
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values))
        train[f] = lbl.transform(list(train[f].values))

train.shape

(3998, 33)

In [7]:
train.head()

,Salary,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,...,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience
0,420000,0,1990,84.3,43,2007,95.8,47,1141,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.9737,0.8128,0.5269,1.35490,-0.4455
1,500000,1,1989,85.4,60,2007,85.0,77,5807,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,-0.7335,0.3789,1.2396,-0.10760,0.8637
2,325000,0,1992,85.0,60,2010,68.2,77,64,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.2718,1.7109,0.1637,-0.86820,0.6721
3,1100000,1,1989,85.6,60,2007,83.6,77,6920,1,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.0464,0.3448,-0.3440,-0.40780,-0.9194
4,200000,1,1991,78.0,60,2008,76.8,77,11368,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,-0.8810,-0.2793,-1.0697,0.09163,-0.1295


In [8]:
test = train.head(500)
train = train.tail(len(train) - 500)
train.head()

,Salary,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,...,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience
500,2600000,1,1989,86.6,0,2007,69.60,2,11467,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.3555,0.9688,0.0914,-0.87780,-0.7615
501,385000,0,1991,89.0,0,2008,94.16,2,3076,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,0.0464,1.3198,-0.4891,-1.11277,-0.2875
502,240000,1,1990,48.0,224,2008,51.40,284,8195,1,...,405.636364,406.876596,516.000000,350.173797,349.666667,-0.5236,-1.1196,0.1637,0.01920,-1.0872
503,260000,1,1990,53.3,224,2007,68.40,284,4501,2,...,315.000000,406.876596,433.037267,350.173797,349.666667,-1.0208,0.0459,-0.1437,1.41360,0.2889
504,460000,1,1990,86.0,119,2008,84.00,249,5284,2,...,405.636364,406.876596,433.037267,350.173797,349.666667,1.1283,1.5928,1.3977,1.00240,-1.1169


In [9]:
target = train.Salary.values
test_target = test.Salary.values

train = train.drop(['Salary'], axis =1) 
test = test.drop(['Salary'], axis =1) 

#X1 = sm.add_constant(train)
est = sm.OLS(target, train).fit()

est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.707
Model:                            OLS   Adj. R-squared (uncentered):              0.704
Method:                 Least Squares   F-statistic:                              269.7
Date:                Thu, 16 Apr 2020   Prob (F-statistic):                        0.00
Time:                        00:24:59   Log-Likelihood:                         -47731.
No. Observations:                3498   AIC:                                  9.552e+04
Df Residuals:                    3467   BIC:                                  9.572e+04
Df Model:                          31                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Gender                 2.344e+04   8569.263      2.735      0.006    6634.313    4.02e+04
DOB                   -1607.4874   4120.290     -0.390      0.696   -9685.927    6470.952
10percentage           1027.2947    503.308      2.041      0.041      40.485    2014.105
10board                 -29.3121     74.175     -0.395      0.693    -174.744     116.119
12graduation           1671.3564   4082.726      0.409      0.682   -6333.433    9676.146
12percentage           1339.1083    439.824      3.045      0.002     476.768    2201.449
12board                -145.4974     57.353     -2.537      0.011    -257.946     -33.049
CollegeID                -1.7856      0.370     -4.829      0.000      -2.511      -1.061
CollegeTier           -7.755e+04   1.44e+04     -5.394      0.000   -1.06e+05   -4.94e+04
Degree                 1910.2279   5089.531      0.375      0.707   -8068.552    1.19e+04
Specialization         -325.9271    388.239     -0.840      0.401   -1087.127     435.273
collegeGPA             1445.6427    485.996      2.975      0.003     492.776    2398.509
CollegeCityID            -1.7856      0.370     -4.829      0.000      -2.511      -1.061
CollegeCityTier       -3710.1826   7782.630     -0.477      0.634    -1.9e+04    1.15e+04
CollegeState           -249.0962    510.334     -0.488      0.626   -1249.681     751.489
GraduationYear           26.1156    102.155      0.256      0.798    -174.174     226.405
English                  97.3938     40.017      2.434      0.015      18.935     175.853
Logical                  14.4114     50.847      0.283      0.777     -85.282     114.104
Quant                   129.9202     36.482      3.561      0.000      58.393     201.448
Domain                 8.827e+04   2.38e+04      3.710      0.000    4.16e+04    1.35e+05
ComputerProgramming      -0.7354     54.085     -0.014      0.989    -106.778     105.307
ElectronicsAndSemicon  -236.0522    106.141     -2.224      0.026    -444.158     -27.947
ComputerScience        -190.7747     97.065     -1.965      0.049    -381.085      -0.465
MechanicalEngg           22.4500    188.039      0.119      0.905    -346.229     391.129
ElectricalEngg          -55.3539    181.476     -0.305      0.760    -411.165     300.458
TelecomEngg              79.6809    149.794      0.532      0.595    -214.013     373.375
CivilEngg               -92.8665    448.804     -0.207      0.836    -972.813     787.080
conscientiousness     -1.818e+04   4201.180     -4.328      0.000   -2.64e+04   -9944.476
agreeableness          1.129e+04   5130.647      2.200      0.028    1227.347    2.13e+04
extraversion           4767.7875   4298.752      1.109      0.267   -3660.554    1.32e+04
nueroticism   

In [145]:
test.head()

,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,Degree,...,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience
0,0,1990,84.3,43,2007,95.8,47,1141,2,0,...,405,406,433,350,349,0.9737,0.8128,0.5269,1.35490,-0.4455
1,1,1989,85.4,60,2007,85.0,77,5807,2,0,...,405,406,433,350,349,-0.7335,0.3789,1.2396,-0.10760,0.8637
2,0,1992,85.0,60,2010,68.2,77,64,2,0,...,405,406,433,350,349,0.2718,1.7109,0.1637,-0.86820,0.6721
3,1,1989,85.6,60,2007,83.6,77,6920,1,0,...,405,406,433,350,349,0.0464,0.3448,-0.3440,-0.40780,-0.9194
4,1,1991,78.0,60,2008,76.8,77,11368,2,0,...,405,406,433,350,349,-0.8810,-0.2793,-1.0697,0.09163,-0.1295


In [146]:
train.head()

,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,Degree,...,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience
500,1,1989,86.6,0,2007,69.60,2,11467,2,0,...,405,406,433,350,349,0.3555,0.9688,0.0914,-0.87780,-0.7615
501,0,1991,89.0,0,2008,94.16,2,3076,2,0,...,405,406,433,350,349,0.0464,1.3198,-0.4891,-1.11277,-0.2875
502,1,1990,48.0,224,2008,51.40,284,8195,1,0,...,405,406,516,350,349,-0.5236,-1.1196,0.1637,0.01920,-1.0872
503,1,1990,53.3,224,2007,68.40,284,4501,2,0,...,315,406,433,350,349,-1.0208,0.0459,-0.1437,1.41360,0.2889
504,1,1990,86.0,119,2008,84.00,249,5284,2,0,...,405,406,433,350,349,1.1283,1.5928,1.3977,1.00240,-1.1169


In [147]:
y_pred = est.predict(test)


In [148]:
ans = math.sqrt(mean_squared_error(test_target, y_pred))
ans

163987.0168176544